### Lab2 (a) Model preperation

In [15]:
from resnet20 import ResNetCIFAR
from train_util import train, finetune, test
import torch
import numpy as np
import matplotlib.pyplot as plt

import time

import torchvision.transforms as transforms
import torchvision
import torch.nn as nn
import torch.optim as optim

from FP_layers import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [2]:
net = ResNetCIFAR(num_layers=20, Nbits=None)
net = net.to(device)

In [3]:
# Load the best weight paramters
net.load_state_dict(torch.load("pretrained_model.pt"))
test(net)

Files already downloaded and verified
Test Loss=0.3231, Test accuracy=0.9150


### Lab2 (b) Prune by percentage

In [4]:
def prune_by_percentage(layer, q):
    """
    Pruning the weight paramters by threshold.
    :param q: pruning percentile. 'q' percent of the least 
    significant weight parameters will be pruned.
    """
    with torch.no_grad():
        # Convert the weight of "layer" to numpy array
        layer_weight = layer.weight.detach().cpu().numpy()
        # Compute the q-th percentile of the abs of the converted array
        percentile = np.percentile(np.abs(layer_weight.flatten()), q)
        # Generate a binary mask same shape as weight to decide which element to prune
        masked_obj = np.ma.masked_greater_equal(x=np.abs(layer_weight), value=percentile, copy=True)
        mask_int = np.ma.getmask(masked_obj).astype(int)
        # Convert mask to torch tensor and put on GPU
        mask_tensor = torch.tensor(mask_int).to(device)
        # Multiply the weight by mask to perform pruning
        assert mask_int.shape == layer_weight.shape
        # layer.weight.data = mask_tensor * layer.weight.data
        layer.weight.data = layer.weight.data.clone().detach().requires_grad_(True) * mask_tensor
    return
    

In [5]:
q_list = np.array([0.3, 0.5, 0.7]) * 100
for q_val in q_list:
    net.load_state_dict(torch.load("pretrained_model.pt"))
    for name, layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # change q value
            prune_by_percentage(layer, q=q_val)
            # break
            ## Optional: Check the sparsity you achieve in each layer
            ## Convert the weight of "layer" to numpy array
            np_weight = layer.weight.detach().cpu().numpy()
            ## Count number of zeros
            zeros = sum((np_weight == 0).flatten())
            ## Count number of parameters
            total = len(np_weight.flatten())
            ## Print sparsity
            # print('Sparsity of ' + name + ': '+ str(zeros/total))
            print('Sparsity of %s: %g' % (name, zeros/total))
    test(net)

Sparsity of head_conv.0.conv: 0.300926
Sparsity of body_op.0.conv1.0.conv: 0.299913
Sparsity of body_op.0.conv2.0.conv: 0.299913
Sparsity of body_op.1.conv1.0.conv: 0.299913
Sparsity of body_op.1.conv2.0.conv: 0.299913
Sparsity of body_op.2.conv1.0.conv: 0.299913
Sparsity of body_op.2.conv2.0.conv: 0.299913
Sparsity of body_op.3.conv1.0.conv: 0.30013
Sparsity of body_op.3.conv2.0.conv: 0.300022
Sparsity of body_op.4.conv1.0.conv: 0.300022
Sparsity of body_op.4.conv2.0.conv: 0.300022
Sparsity of body_op.5.conv1.0.conv: 0.300022
Sparsity of body_op.5.conv2.0.conv: 0.300022
Sparsity of body_op.6.conv1.0.conv: 0.300022
Sparsity of body_op.6.conv2.0.conv: 0.299995
Sparsity of body_op.7.conv1.0.conv: 0.299995
Sparsity of body_op.7.conv2.0.conv: 0.299995
Sparsity of body_op.8.conv1.0.conv: 0.299995
Sparsity of body_op.8.conv2.0.conv: 0.299995
Sparsity of final_fc.linear: 0.3
Files already downloaded and verified
Test Loss=0.3699, Test accuracy=0.9027
Sparsity of head_conv.0.conv: 0.5
Sparsity

### Lab2 (c) Finetune pruned model

In [9]:
def finetune_after_prune(net, trainloader, criterion, optimizer, prune=True):
    """
    Finetune the pruned model for a single epoch
    Make sure pruned weights are kept as zero
    """
    # Build a dictionary for the nonzero weights
    weight_mask = {}
    for name, layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Your code here: generate a mask in GPU torch tensor to have 1 for nonzero element and 0 for zero element 
            layer_weight = layer.weight.detach().cpu().numpy()
            mask_obj = np.ma.masked_equal(x=layer_weight, value=0)  # where it's zero, mask value is True
            weight_mask[name] = torch.tensor(1 - np.ma.getmask(mask_obj).astype(int)).to(device)  
            # zero = 0, nonzero = 1
    
    global_steps = 0
    train_loss = 0
    correct = 0
    total = 0
    start = time.time()
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        if prune:
            for name,layer in net.named_modules():
                if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
                    # Your code here: Use weight_mask to make sure zero elements remains zero                    
                    layer.weight.data = layer.weight.data.clone().detach().requires_grad_(True) * weight_mask[name]
                    
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        global_steps += 1

        if global_steps % 50 == 0:
            end = time.time()
            batch_size = 256
            num_examples_per_second = 50 * batch_size / (end - start)
            print("[Step=%d]\tLoss=%.4f\tacc=%.4f\t%.1f examples/second"
                 % (global_steps, train_loss / (batch_idx + 1), (correct / total), num_examples_per_second))
            start = time.time()

In [10]:
# Get pruned model
net.load_state_dict(torch.load("pretrained_model.pt"))
for name, layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        prune_by_percentage(layer, q=70.0)

# Training setup, do not change
batch_size = 256
lr = 0.002
reg = 1e-4

print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=16)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.875, weight_decay=reg, nesterov=False)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# Model finetuning
test_acc_lst = []
n_epochs = 20
for epoch in range(n_epochs):
    print('\nEpoch: %d' % epoch)
    net.train()
    finetune_after_prune(net, trainloader, criterion, optimizer)
    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))
    test_acc_lst.append(val_acc)
    if val_acc > best_acc:
        best_acc = val_acc
        print("Saving...")
        torch.save(net.state_dict(), "net_after_finetune.pt")

In [ ]:
fig, ax = plt.subplots(1, 1)
xx = range(n_epochs)
ax.plot(xx, test_acc_lst, label='last test accuracy: %g' % test_acc_lst[-1])
ax.set_xlabel('Epochs')
ax.set_ylabel('Accuracy')
ax.set_title('lab2c Accuracy vs Epochs\nFine-tune Pruning')

plt.savefig('lab2c.pdf', dpi=500, bbox_inches='tight')

In [ ]:
# Check sparsity of the finetuned model, make sure it's not changed
net.load_state_dict(torch.load("net_after_finetune.pt"))

for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # Your code here:
        ## Convert the weight of "layer" to numpy array
        np_weight = layer.weight.detach().cpu().numpy()
        ## Count number of zeros
        zeros = sum((np_weight == 0).flatten())
        ## Count number of parameters
        total = len(np_weight.flatten())
        # Print sparsity
        print('Sparsity of %s: %g' % (name, zeros/total))   
        
test(net)

### Lab2 (d) Iterative pruning

In [ ]:
net.load_state_dict(torch.load("pretrained_model.pt"))
best_acc = 0.
n_epochs = 20

test_acc_iter_prune = []

for epoch in range(n_epochs):
    print('\nEpoch: %d' % epoch)
    
    net.train()
    if epoch < 10:
        for name,layer in net.named_modules():
            if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
                # Increase model sparsity
                q = (epoch + 1) * 7
                # Linearly increase the pruning percentage for 10 epochs until reaching 70% in the final epoch
                prune_by_percentage(layer, q=q)
    if epoch < 9:
        finetune_after_prune(net, trainloader, criterion, optimizer, prune=False)
    else: # starts from epoch==9, where q = 70%
        finetune_after_prune(net, trainloader, criterion, optimizer)
    
    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    
    test_acc_iter_prune.append(val_acc)
        
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))
    
    if epoch>=10:
        if val_acc > best_acc:
            best_acc = val_acc
            print("Saving...")
            torch.save(net.state_dict(), "net_after_iterative_prune.pt")

In [ ]:
# Check sparsity of the final model, make sure it's 70%
net.load_state_dict(torch.load("net_after_iterative_prune.pt"))

for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # Your code here: can copy from previous question
        ## Convert the weight of "layer" to numpy array
        np_weight = layer.weight.detach().cpu().numpy()
        ## Count number of zeros
        zeros = sum((np_weight == 0).flatten())
        ## Count number of parameters
        total = len(np_weight.flatten())
        # Print sparsity
        print('Sparsity of %s: %g' % (name, zeros/total))   
        
        
test(net)

In [ ]:
fig, ax = plt.subplots(1, 1)
xx = range(n_epochs)
ax.plot(xx, test_acc_iter_prune, label='max test accuracy: %g' % test_acc_iter_prune[-1])
ax.set_xlabel('Epochs')
ax.set_ylabel('Accuracy')
ax.set_title('lab2d Accuracy vs Epochs\nIterative Pruning')

plt.savefig('lab2d.pdf', dpi=500, bbox_inches='tight')

### Lab2 (e) Global iterative pruning

In [ ]:
def global_prune_by_percentage(net, q=70.0):
    """
    Pruning the weight paramters by threshold.
    :param q: pruning percentile. 'q' percent of the least 
    significant weight parameters will be pruned.
    """
    # A list to gather all the weights
    flattened_weights = []
    # Find global pruning threshold
    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Convert weight to numpy
            # Flatten the weight and append to flattened_weights
            flattened_weights.append(layer.weight.detach().cpu().numpy().flatten())
    
    # Concate all weights into a np array
    flattened_weights = np.concatenate(flattened_weights)
    # Find global pruning threshold
    threshold = np.percentile(пр.abs(flattened_weights.flatten()), q)
    
    # Apply pruning threshold to all layers
    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Convert weight to numpy
            layer_weight = layer.weight.detach().cpu().numpy()
            
            # Generate a binary mask same shape as weight to decide which element to prune
            masked_obj = np.ma.masked_greater_equal(x=np.abs(layer_weight), value=threshold, copy=True)
            mask_int = np.ma.getmask(masked_obj).astype(int)
            # Convert mask to torch tensor and put on GPU
            mask_tensor = torch.tensor(mask_ine).to(device)
            # Multiply the weight by mask to perform pruning
            layer.weight.data = layer.weight.data.clone().detach().requires_grad_(True) * mask_tensor

In [ ]:
net.load_state_dict(torch.load("pretrained_model.pt"))
best_acc = 0.

test_acc_global_prune = []

for epoch in range(20):
    print('\nEpoch: %d' % epoch)
    # q = (epoch + 1) * 8
    q = (epoch + 1) * 7
    
    net.train()
    # Increase model sparsity
    if epoch < 10:
        global_prune_by_percentage(net, q=q)
    if epoch < 9:
        finetune_after_prune(net, trainloader, criterion, optimizer, prune=False)
    else:
        finetune_after_prune(net, trainloader, criterion, optimizer)
    
    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))
    
    test_acc_global_prune.append(val_acc)
    
    if epoch >= 10:
        if val_acc > best_acc:
            best_acc = val_acc
            print("Saving...")
            torch.save(net.state_dict(), "net_after_global_iterative_prune.pt")

In [ ]:
net.load_state_dict(torch.load("net_after_global_iterative_prune.pt"))

zeros_sum = 0
total_sum = 0
for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # Your code here:
        # Convert the weight of "layer" to numpy array
        np_weight = layer.weight.detach().cpu().numpy()
        # Count number of zeros
        zeros = sum((np_weight == 0).flatten())
        # Count number of parameters
        total = len(np_weight.flatten())
        zeros_sum += zeros
        total_sum += total
        print('Sparsity of %s: %g' % (name, zeros/total))   
print('Total sparsity of: %g' % float(zeros_sum/total_sum))
test(net)

In [ ]:
fig, ax = plt.subplots(1, 1)
xx = range(n_epochs)
ax.plot(xx, test_acc_global_prune, label='last test accuracy: %g' % test_acc_global_prune[-1])
ax.set_xlabel('Epochs')
ax.set_ylabel('Accuracy')
ax.set_title('lab2e Accuracy vs Epochs\nGlobal Iterative Pruning')

plt.savefig('lab2e.pdf', dpi=500, bbox_inches='tight')

### Lab 3 (b) and (c): Fixed-point quantization

In [ ]:
# Define quantized model and load weight
Nbits = 4 #Change this value to finish (b) and (c)

net = ResNetCIFAR(num_layers=20, Nbits=Nbits)
net = net.to(device)
net.load_state_dict(torch.load("pretrained_model.pt"))
test(net)


In [ ]:
# Quantized model finetuning
finetune(net, epochs=20, batch_size=256, lr=0.002, reg=1e-4)   

# Load the model with best accuracy
net.load_state_dict(torch.load("quantized_net_after_finetune.pt"))
test(net)

### Lab3 (d) Quantize pruned model

In [ ]:
# Define quantized model and load weight
Nbits = 3 #Change this value to finish (d)

net = ResNetCIFAR(num_layers=20, Nbits=Nbits)
net = net.to(device)
net.load_state_dict(torch.load("net_after_global_iterative_prune.pt"))
test(net)


In [ ]:
# Quantized model finetuning
finetune(net, epochs=20, batch_size=256, lr=0.002, reg=1e-4)

# Load the model with best accuracy
net.load_state_dict(torch.load("quantized_net_after_finetune.pt"))
test(net)

### Lab3 (e) Symmetric quantization
#### Implement symmetric quantization in FP_layers.py, and repeat the process in (b)

In [ ]:
# check the performance of symmetric quantization with 6, 5, 4, 3, 2 bits